In [36]:
# Importing dependecies:
import pandas as pd
import numpy as np
import re
import nltk
from nltk import sent_tokenize, word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import seaborn as sns

import gensim.models as gs
from gensim.models.phrases import Phrases, Phraser

from numpy import array
import keras
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM, GlobalMaxPooling1D, Dense, Conv2D
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
import tensorflow as tf

# import torch
# import torch.nn as nn
# import torch.nn.functional as F

# Configuring Notebook environment:
sns.set()
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

plt.rcParams['figure.figsize'] = (10.0, 7.5)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [39]:
df = pd.read_csv('data/strings/df_clean.csv', index_col=0)
df.head()

,title,ingredients,instructions,ingredients_vector,instructions_vector
p3pKOD6jIHEcjf20CCXohP8uqkG5dGi,grammie hamblets deviled crab,celery finely chopped green pepper finely chop...,toss ingredients lightly spoon buttered baking...,"['celery', 'finely', 'chopped', 'green', 'pepp...","['toss', 'ingredients', 'lightly', 'spoon', 'b..."
S7aeOIrsrgT0jLP32jKGg4j.o9zi2DO,infineon raceway baked beans,skirt steak cut inch dicekosher salt fresh cra...,watch make recipe sprinkle steak salt pepper s...,"['skirt', 'steak', 'cut', 'inch', 'dicekosher'...","['watch', 'make', 'recipe', 'sprinkle', 'steak..."
o9MItV9txfoPsUQ4v8b0vh1.VdjwfsK,southwestern black bean dip,cups dried black beans picked rinsed cups wate...,saucepan let beans soak enough cold water cove...,"['cups', 'dried', 'black', 'beans', 'picked', ...","['saucepan', 'let', 'beans', 'soak', 'enough',..."
5l1yTSYFifF/M2dfbD6DX28WWQpLWNK,sour cream noodle bake,ground chuckone tomato sauce saltfreshly groun...,watch make recipe preheat oven degrees f brown...,"['ground', 'chuckone', 'tomato', 'sauce', 'sal...","['watch', 'make', 'recipe', 'preheat', 'oven',..."
kRBQSWtqYWqtkb34FGeenBSbC32gIdO,sushi renovation,rice brown mediumgrain cookedcup quinoacup swe...,special equipment sushi mat cook brown rice qu...,"['rice', 'brown', 'mediumgrain', 'cookedcup', ...","['special', 'equipment', 'sushi', 'mat', 'cook..."


## Word2Vec:

In [30]:
model_ingredients = gs.Word2Vec(df['ingredients_vector'], min_count=1, size= 25, workers=3, window=3, sg=1)
model_instructions = gs.Word2Vec(df['instructions_vector'], min_count=1, size= 25, workers=3, window=3, sg=1)

In [61]:
# print(model_ingredients)
# data = model_instructions.most_similar('beef')
# print(data)

In [40]:
X = df.drop(['title', 'ingredients', 'instructions'], axis=1)
y = df['title'].values
print(X.shape, y.shape)

(59612, 2) (59612,)


In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
print('Train shape:', X_train.shape, y_train.shape)
print('Test shape:', X_test.shape, y_test.shape)

Train shape: (47689, 2) (47689,)
Test shape: (11923, 2) (11923,)


In [65]:
# The unique characters in the file
vocab = sorted(set(X_train))
print ('{} unique characters'.format(len(vocab)))

2 unique characters


In [63]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(X_train)}
print(char2idx)

{'ingredients_vector': 0, 'instructions_vector': 1}


In [56]:
char2idx = {u:x for i, u in enumerate(x)) x for x in df['ingredients_vector']}
idx2char = np.array(df['ingredients_vector'])

text_as_int = np.array([char2idx[c] for c in df['ingredients_vector']])

SyntaxError: invalid syntax (<ipython-input-56-50584ad557c2>, line 1)

In [51]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  "['celery', 'finely', 'chopped', 'green', 'pepper', 'finely', 'chopped', 'finely', 'sliced', 'green', 'onions', 'chopped', 'parsley', 'crabmeat', 'cups', 'coarsely', 'crushed', 'cracker', 'crumbs', 'salt', 'teaspoons', 'dry', 'mustarddash', 'hot', 'sauce', 'heavy', 'cream', 'melted', 'butter']": 56784,
  "['skirt', 'steak', 'cut', 'inch', 'dicekosher', 'salt', 'fresh', 'cracked', 'black', 'pepper', 'slices', 'thickcut', 'applewood', 'smoked', 'bacon', 'inch', 'diced', 'cups', 'inch', 'diced', 'red', 'onion', 'seeded', 'finely', 'diced', 'jalapenos', 'tablespoons', 'minced', 'garlic', 'kosher', 'salt', 'teaspoons', 'fresh', 'cracked', 'black', 'pepperapple', 'cider', 'vinegar', 'best', 'quality', 'cannellini', 'beans', 'liquid', 'lima', 'beans', 'liquid', 'kidney', 'beans', 'liquid', 'molasses', 'tablespoons', 'dark', 'brown', 'sugar', 'tablespoons', 'soy', 'sauce', 'ketchupcrusty', 'bread', 'serving']":   1,
  "['cups', 'dried', 'black', 'beans', 'picked', 'rinsed', 'cups', 'water'

In [53]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(df['ingredients_vector'][:13]), text_as_int[:13]))

p3pKOD6jIHEcjf20CCXohP8uqkG5dGi    ['celery', 'finely', 'chopped', 'green', 'pepp...
S7aeOIrsrgT0jLP32jKGg4j.o9zi2DO    ['skirt', 'steak', 'cut', 'inch', 'dicekosher'...
o9MItV9txfoPsUQ4v8b0vh1.VdjwfsK    ['cups', 'dried', 'black', 'beans', 'picked', ...
5l1yTSYFifF/M2dfbD6DX28WWQpLWNK    ['ground', 'chuckone', 'tomato', 'sauce', 'sal...
kRBQSWtqYWqtkb34FGeenBSbC32gIdO    ['rice', 'brown', 'mediumgrain', 'cookedcup', ...
PmoGGX9RC5zbMJXtZH1SKzWKLFX4Aii    ['extravirgin', 'olive', 'oil', 'baby', 'itali...
g2RYP1spIUlAYsytDMsdfLNQEOnLUrO    ['lightly', 'crumbled', 'saffron', 'threads', ...
94NWE4F4C5NzgH0zYqSdmPM/G55BjKO    ['topsliced', 'hotdog', 'buns', 'fashion', 'to...
HdIbPB/8cgk17hAzbc2jUN4VBn//QKS    ['green', 'red', 'bell', 'peppers', 'olive', '...
mC7Nb4F2S.DMCjZ39kzS/ak9GqEp4iO    ['head', 'cauliflower', 'cut', 'bitesize', 'fl...
QaFfhzgVliz840HdAnIypaGteQn5q3K    ['cups', 'chicken', 'stock', 'preferably', 'ho...
2ZlbPBIZEDWBdGFcJ/KhApHyOTYvt.G    ['cups', 'sweet', 'chili', 'sa

In [42]:
#one-hot encode target column
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

ValueError: invalid literal for int() with base 10: 'simple white sangria'

In [37]:
#create model
model = Sequential()
#add model layers
model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(28,28,1)))
model.add(Conv2D(32, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))

In [23]:
model = Sequential()
model.add(LSTM(256, input_shape=(20, len(df))))
model.add(LSTM(128))
model.add(Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])

ValueError: Input 0 is incompatible with layer lstm_8: expected ndim=3, found ndim=2

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
X_train.shape, y_train.shape

In [ ]:
tokenizer = Tokenizer(num_words=1000)
# x_train = tokenizer.sequences_to_matrix(X_train, mode='binary')
# x_test = tokenizer.sequences_to_matrix(X_test, mode='binary')

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
num_classes = 2
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(y_train.shape)
print(y_test.shape)

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=3)